In [1]:
import os
from os import path as op
import numpy as np
from scipy import stats
from scipy import linalg
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist
import vlgp
from vlgp import util, simulation

In [2]:
# Set dimensions and simulation parameters
K1 = 500  # Number of observations for Group 1
K2 = 800  # Number of observations for Group 2
K = K1 + K2  # Total number of observations
D = 6  # Total latent dimensions
T = 250  # Number of time points
t = np.linspace(0, 2, T)  # Time intervals

d_s = 2  # Shared latent dimension
d_1 = 2  # Independent latent dimension for group 1
d_2 = 2  # Independent latent dimension for group 2

rho = 1.0  # Scale for GP kernel
l = 2.0  # Length scale for GP kernel
nu = 0.1  # Noise variance for observations

# Set factor loadings
A_1 = np.random.randn(K1, d_1)
A_2 = np.random.randn(K2, d_2)
A_s1 = np.random.randn(K1, d_s)
A_s2 = np.random.randn(K2, d_s)

A = np.block([[A_s1, A_1, np.zeros((K1, d_2))],
              [A_s2, np.zeros((K2, d_1)), A_2]])  # Group 2

def kernel_function(t1, t2, rho, l):
    """Squared exponential kernel."""
    dist_sq = cdist(t1.reshape(-1, 1), t2.reshape(-1, 1), metric='sqeuclidean')
    return rho * np.exp(-dist_sq / (2 * l ** 2))

K_t = kernel_function(t, t, rho, l)

z_shared = np.random.multivariate_normal(np.zeros(T), K_t, size=d_s).T
z_1 = np.random.multivariate_normal(np.zeros(T), K_t, size=d_1).T
z_2 = np.random.multivariate_normal(np.zeros(T), K_t, size=d_2).T

Z = np.hstack([z_shared, z_1, z_2])
Z

array([[-1.18876669, -0.79731037,  2.24790178,  0.30609292,  1.45868638,
        -0.069562  ],
       [-1.18810899, -0.79546104,  2.24371928,  0.30346968,  1.4577649 ,
        -0.06896218],
       [-1.18743995, -0.79361418,  2.23949058,  0.30086935,  1.45686859,
        -0.06838684],
       ...,
       [-1.07073471, -0.24673677,  0.76868683,  0.62589043,  1.51516996,
        -0.02588428],
       [-1.07095014, -0.24348184,  0.76434735,  0.6307363 ,  1.5137523 ,
        -0.02382317],
       [-1.07116781, -0.24021812,  0.76004   ,  0.63559342,  1.51229686,
        -0.02173806]])

In [3]:
ntrial = 10  # Number of trials
nbin = 250  # Number of bins per trial to match T=250 as previously set
dim = 6  # Number latent dimensions
Z_cut = Z[:(Z.shape[0] // nbin) * nbin]
trials = [{'ID': i, 'y': Z_cut[i * nbin: (i + 1) * nbin].reshape(nbin, dim)} for i in range(Z_cut.shape[0] // nbin)]
trials

[{'ID': 0,
  'y': array([[-1.18876669, -0.79731037,  2.24790178,  0.30609292,  1.45868638,
          -0.069562  ],
         [-1.18810899, -0.79546104,  2.24371928,  0.30346968,  1.4577649 ,
          -0.06896218],
         [-1.18743995, -0.79361418,  2.23949058,  0.30086935,  1.45686859,
          -0.06838684],
         ...,
         [-1.07073471, -0.24673677,  0.76868683,  0.62589043,  1.51516996,
          -0.02588428],
         [-1.07095014, -0.24348184,  0.76434735,  0.6307363 ,  1.5137523 ,
          -0.02382317],
         [-1.07116781, -0.24021812,  0.76004   ,  0.63559342,  1.51229686,
          -0.02173806]])}]

In [4]:
#np.random.seed(0)

fit = vlgp.fit(
    trials,  
    n_factors=3,  # dimensionality of latent process
    max_iter=20,  # maximum number of iterations
    min_iter=10  # minimum number of iterations
)

Initializing
Initialized
Fitting
Iteration    1, E-step 0.12s, M-step 0.03s
Iteration    2, E-step 0.12s, M-step 0.03s
Iteration    3, E-step 0.11s, M-step 0.03s
Iteration    4, E-step 0.11s, M-step 0.04s
Iteration    5, E-step 0.12s, M-step 0.03s
Iteration    6, E-step 0.12s, M-step 0.03s
Iteration    7, E-step 0.12s, M-step 0.03s
Iteration    8, E-step 0.12s, M-step 0.03s
Iteration    9, E-step 0.11s, M-step 0.03s
Iteration   10, E-step 0.14s, M-step 0.03s
Iteration   11, E-step 0.13s, M-step 0.04s
Iteration   12, E-step 0.12s, M-step 0.03s
Iteration   13, E-step 0.12s, M-step 0.03s
Iteration   14, E-step 0.11s, M-step 0.03s
Iteration   15, E-step 0.12s, M-step 0.03s
Iteration   16, E-step 0.12s, M-step 0.03s
Iteration   17, E-step 0.11s, M-step 0.03s
Iteration   18, E-step 0.12s, M-step 0.03s
Iteration   19, E-step 0.12s, M-step 0.03s
Iteration   20, E-step 0.12s, M-step 0.03s
Inferring
0.14s
Done


In [5]:
fit

{'trials': [{'ID': 0,
   'y': array([[-1.18876669, -0.79731037,  2.24790178,  0.30609292,  1.45868638,
           -0.069562  ],
          [-1.18810899, -0.79546104,  2.24371928,  0.30346968,  1.4577649 ,
           -0.06896218],
          [-1.18743995, -0.79361418,  2.23949058,  0.30086935,  1.45686859,
           -0.06838684],
          ...,
          [-1.07073471, -0.24673677,  0.76868683,  0.62589043,  1.51516996,
           -0.02588428],
          [-1.07095014, -0.24348184,  0.76434735,  0.6307363 ,  1.5137523 ,
           -0.02382317],
          [-1.07116781, -0.24021812,  0.76004   ,  0.63559342,  1.51229686,
           -0.02173806]]),
   'mu': array([[-2.52009797e+46,  4.71308563e+44,  1.66585801e+45],
          [-2.80584472e+46,  5.09610679e+44,  2.78974308e+45],
          [-3.08533311e+46,  5.47240147e+44,  3.86565001e+45],
          [-3.35495135e+46,  5.83808361e+44,  4.86686550e+45],
          [-3.61146295e+46,  6.18958535e+44,  5.77240160e+45],
          [-3.85211966e+46,  